In [1]:
import tensorflow as tf
from random import random
from numpy import array
from numpy import cumsum
import numpy as np
from nltk import word_tokenize, pos_tag, ne_chunk
from nltk import conlltags2tree, tree2conlltags
import nltk
from nltk.tree import Tree
from nltk.tag.mapping import map_tag
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense,Flatten
from keras.layers import TimeDistributed
from keras.layers import Bidirectional
from keras.layers import Activation
from keras.models import Model, Input
from keras.layers.merge import add
from keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional, Lambda

/share/apps/python3/3.6.3/intel/lib/python3.6/site-packages/h5py-2.7.1-py3.6-linux-x86_64.egg/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def scrubscrub(file):
    #filename here
    source_file = file  
    #make output file
   
    #open file for reading
    arr = []
    arrtag = []
    
    with open(source_file, "r") as data:
        #loop through everyline
        String = []
        tags = []
        for line in data:
            if  line in ['\n', '\r\n']:
                #print(String)
                s = " "
                arr.append((String))
                String = []
                arrtag.append(tags)
                tags = []
            else:
                arrayline = line.strip().split(" ")
                #print(arrayline)
                word = arrayline[0]
#                 if word in string.punctuation:
#                     continue
                tag = arrayline[-1]
                String.append(word)
                tags.append(tag)
    #print(arr)
    return arr, arrtag
            #write the lines out to the output files
 



In [3]:

train, traintag = scrubscrub("eng.train")
val, valtag = scrubscrub("eng.testa")
test, testtag = scrubscrub("eng.testb")

In [4]:
#train

In [5]:
#traintag


In [6]:
#pad every setence with padwords
max_len = 100
def convertx(train): 
    sentences = train
    
    X = [s for s in sentences]
    new_X = []
    for seq in X:
        new_seq = []
        for i in range(max_len):
            try:
                new_seq.append(seq[i])
            except:
                new_seq.append("PADword")
        new_X.append(new_seq)
    return new_X
new_Train = convertx(train)
new_Val = convertx(val)
new_Test = convertx(test)
print(new_Train[15])

['Sheep', 'have', 'long', 'been', 'known', 'to', 'contract', 'scrapie', ',', 'a', 'brain-wasting', 'disease', 'similar', 'to', 'BSE', 'which', 'is', 'believed', 'to', 'have', 'been', 'transferred', 'to', 'cattle', 'through', 'feed', 'containing', 'animal', 'waste', '.', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PA

In [7]:
#convert the tags of datasets to one hot vectors with 1 representing the ocrrect tag
tags2index={'O': 0, 'I-ORG': 1, 'I-MISC': 2, 'I-PER': 3, 'I-LOC':4, 'B-LOC': 5, 'B-MISC': 6, 'B-ORG': 7,'B-PER':8}
def onehot(a):
    arr = []
    for i in range(0,9):
        if i == a:
            arr.append(1)
        else: 
            arr.append(0)
    return arr        
from keras.preprocessing.sequence import pad_sequences
def converty(dataset):
    y = [[onehot(tags2index[tag]) for tag in s] for s in dataset]
    y = pad_sequences(maxlen=max_len, sequences=y, padding="post", value=onehot(tags2index["O"]))
    #print(y[15])
    #print(dataset[15])
    return y
new_Traintag = converty(traintag)
new_Valtag = converty(valtag)
new_Testtag = converty(testtag)
print(len(new_Traintag),len(new_Valtag),len(new_Testtag) )

14987 3466 3684


In [8]:

batch_size = 256
import tensorflow as tf
import tensorflow_hub as hub
from keras import backend as K
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
K.set_session(sess)



W0413 17:17:30.958376 47532468810368 __init__.py:56] Some hub symbols are not available because TensorFlow version is less than 1.14


In [68]:
# use elmo embeddings
elmo_model = hub.Module("https://tfhub.dev/google/elmo/2", trainable=True)
sess.run(tf.global_variables_initializer())
sess.run(tf.tables_initializer())

In [10]:
def ElmoEmbedding(x):
    return elmo_model(inputs={
                            "tokens": tf.squeeze(tf.cast(x, tf.string)),
                            "sequence_len": tf.constant(batch_size*[max_len])
                      },
                      signature="tokens",
                      as_dict=True)["elmo"]

In [11]:
from keras.models import Model, Input
from keras.layers.merge import add
from keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional, Lambda

In [12]:
#model definition
n_tags = 9
input_text = Input(shape=(max_len,), dtype=tf.string)
embedding = Lambda(ElmoEmbedding, output_shape=(max_len, 1024))(input_text)
x = Bidirectional(LSTM(units=1024, return_sequences=True,
                       recurrent_dropout=0.2, dropout=0.2))(embedding)
x_rnn = Bidirectional(LSTM(units=1024, return_sequences=True,
                           recurrent_dropout=0.2, dropout=0.2))(x)
x_rnn = Bidirectional(LSTM(units=1024, return_sequences=True))(x_rnn)
x = add([x, x_rnn])  # residual connection to the first biLSTM
x = Dropout(0.5)(x)
x = Dense(64, activation='relu')(x)
x = Dense(64, activation='relu')(x)
out = TimeDistributed(Dense(n_tags, activation="softmax"))(x)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0413 17:17:52.438611 47532468810368 saver.py:1483] Saver not created because there are no variables in the graph to restore


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


W0413 17:17:58.774414 47532468810368 deprecation.py:506] From /home/wj552/pyenv/py3.6.3/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [13]:
#model compile and training
model = Model(input_text, out)
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

In [14]:
%%time
history = model.fit(np.array(new_Train[:14848]), new_Traintag[:14848], validation_data=(np.array(new_Val[:3328]), new_Valtag[:3328]),
                    batch_size=batch_size, epochs=25, verbose=1)

Instructions for updating:
Use tf.cast instead.


W0413 17:18:13.522698 47532468810368 deprecation.py:323] From /home/wj552/pyenv/py3.6.3/lib/python3.6/site-packages/tensorflow/python/ops/math_ops.py:3066: to_int32 (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.cast instead.


Train on 14848 samples, validate on 3328 samples
Epoch 1/20
14848/14848 [==============================] - 234s 16ms/step - loss: 0.0931 - acc: 0.9694 - val_loss: 0.0191 - val_acc: 0.9946
Epoch 2/20
14848/14848 [==============================] - 214s 14ms/step - loss: 0.0133 - acc: 0.9959 - val_loss: 0.0118 - val_acc: 0.9965
Epoch 3/20
14848/14848 [==============================] - 215s 14ms/step - loss: 0.0093 - acc: 0.9972 - val_loss: 0.0108 - val_acc: 0.9971
Epoch 4/20
14848/14848 [==============================] - 215s 14ms/step - loss: 0.0075 - acc: 0.9977 - val_loss: 0.0101 - val_acc: 0.9974
Epoch 5/20
14848/14848 [==============================] - 216s 15ms/step - loss: 0.0062 - acc: 0.9981 - val_loss: 0.0090 - val_acc: 0.9976
Epoch 6/20
14848/14848 [==============================] - 216s 15ms/step - loss: 0.0050 - acc: 0.9985 - val_loss: 0.0086 - val_acc: 0.9978
Epoch 7/20
14848/14848 [==============================] - 216s 15ms/step - loss: 0.0043 - acc: 0.9987 - val_loss: 0.0

In [37]:
from seqeval.metrics import precision_score, recall_score, f1_score, classification_report
new_Test = new_Test[:3584]
test_pred = model.predict(np.array(new_Test), verbose=1,batch_size= batch_size)


3584/3584 [==============================] - 26s 7ms/step


I used a seqeval evaluation of f1 and such at first but then i reaized i can use conlleval.

In [38]:
#convert the softmax vector of size 9 into the correspoding tags
idx2tag = dict([(value, key) for key, value in tags2index.items()])

def pred2label(pred):
    out = []
    for sent in pred:
        out_i = []
        for pmatrix in sent:
            p_i = np.argmax(pmatrix)
            #print(p_i)
            out_i.append(idx2tag[p_i].replace("PADword", "O"))
        out.append(out_i)
    return out

def test2label(pred):
    out = []
    for pred_i in pred:
        out_i = []
        for p in pred_i:
            p_i = np.argmax(p)
            out_i.append(idx2tag[p_i].replace("PADword", "O"))
        out.append(out_i)
    return out
#test_pred
pred_labels = pred2label(test_pred)
test_labels = test2label(new_Testtag[:3584])
#print(pred_labels)
#print(new_Testtag)

In [59]:
# with open('predy.txt', 'w') as f:
#     for item in pred_labels:
#         for tag in item:
#             f.write("%s " % tag)
#     f.write("\n")
# with open('testY.txt', 'w') as f:
#     for item in test_labels:
#          for tag in item:
#             f.write("%s " % tag)
#     f.write("\n")

In [51]:
print("F1-score: {:.1%}".format(f1_score(test_labels, pred_labels)))
score = f1_score(test_labels, pred_labels)

F1-score: 89.5%


In [52]:
print(classification_report(test_labels, pred_labels))

             precision    recall  f1-score   support

        ORG       0.87      0.89      0.88      1579
        PER       0.97      0.95      0.96      1577
        LOC       0.92      0.91      0.92      1630
       MISC       0.72      0.78      0.75       680

avg / total       0.89      0.90      0.90      5466



In [53]:
# serialize model to YAML
model_yaml = model.to_yaml()
with open("model.yaml", "w") as yaml_file:
    yaml_file.write(model_yaml)
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")

Saved model to disk


In [54]:
pred_labels

[['O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O'],
 ['O',
  'O',
  'I-LOC',
  'O',
  'O',
  'O',
  'O',
  'I-LOC',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O

In [55]:
test_labels


[['O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O'],
 ['O',
  'O',
  'I-LOC',
  'O',
  'O',
  'O',
  'O',
  'I-PER',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O

n_tags = 9
input_text = Input(shape=(max_len,), dtype=tf.string)
embedding = Lambda(ElmoEmbedding, output_shape=(max_len, 1024))(input_text)
x = Bidirectional(LSTM(units=512, return_sequences=True,
                       recurrent_dropout=0.2, dropout=0.2))(embedding)
x_rnn = Bidirectional(LSTM(units=512, return_sequences=True,
                           recurrent_dropout=0.2, dropout=0.2))(x)
x_rnn = Bidirectional(LSTM(units=512, return_sequences=True,
                           recurrent_dropout=0.4, dropout=0.4))(x_rnn)
x = add([x, x_rnn])  # residual connection to the first biLSTM
x = Dropout(0.5)(x)
out = TimeDistributed(Dense(n_tags, activation="softmax"))(x)

F1-score: 84.2%
class report:
       precision    recall  f1-score   support

       MISC       0.56      0.75      0.64       680
        ORG       0.80      0.79      0.80      1579
        LOC       0.87      0.90      0.88      1630
        PER       0.95      0.94      0.95      1577

avg / total       0.83      0.86      0.85      5466


n_tags = 9
input_text = Input(shape=(max_len,), dtype=tf.string)
embedding = Lambda(ElmoEmbedding, output_shape=(max_len, 1024))(input_text)
x = Bidirectional(LSTM(units=512, return_sequences=True,
                       recurrent_dropout=0.2, dropout=0.2))(embedding)
x_rnn = Bidirectional(LSTM(units=512, return_sequences=True,
                           recurrent_dropout=0.2, dropout=0.2))(x)
x_rnn = Bidirectional(LSTM(units=512, return_sequences=True,
                           recurrent_dropout=0.4, dropout=0.4))(x_rnn)
x = add([x, x_rnn])  # residual connection to the first biLSTM
x = Dropout(0.5)(x)
x = Dense(64, activation='relu')(x)
x = Dense(64, activation='relu')(x)
out = TimeDistributed(Dense(n_tags, activation="softmax"))(x)
F1-score: 84.9%
class report:
             precision    recall  f1-score   support

        ORG       0.78      0.84      0.81      1579
       MISC       0.68      0.70      0.69       680
        PER       0.92      0.94      0.93      1577
        LOC       0.87      0.88      0.88      1630

avg / total       0.84      0.86      0.85      5466


n_tags = 9
input_text = Input(shape=(max_len,), dtype=tf.string)
embedding = Lambda(ElmoEmbedding, output_shape=(max_len, 1024))(input_text)
x = Bidirectional(LSTM(units=512, return_sequences=True,
                       recurrent_dropout=0.2, dropout=0.2))(embedding)
x_rnn = Bidirectional(LSTM(units=512, return_sequences=True,
                           recurrent_dropout=0.2, dropout=0.2))(x)
x_rnn = Bidirectional(LSTM(units=512, return_sequences=True,
                           recurrent_dropout=0.4, dropout=0.4))(x_rnn)
x = add([x, x_rnn])  # residual connection to the first biLSTM
x = Dropout(0.5)(x)
x = Dense(64, activation='relu')(x)
x = Dense(64, activation='relu')(x)
out = TimeDistributed(Dense(n_tags, activation="softmax"))(x)
10 epochs:
F1-score: 88.5%
class report:
             precision    recall  f1-score   support

        PER       0.97      0.94      0.96      1577
        ORG       0.81      0.90      0.85      1579
       MISC       0.73      0.75      0.74       680
        LOC       0.92      0.90      0.91      1630

avg / total       0.88      0.89      0.89      5466

In [66]:
# new_Test
# test_labels
# pred_labels
with open("modelq2.output",'w') as output:
    counter =0
    for i in range(0, len(test_labels)):
            #counter+=1
        for j in range(0,len(test_labels[i])):
            #print(arrtag[counter])
            if(new_Test[i][j]=="PADword"):
                continue
            newarray = [new_Test[i][j],test_labels[i][j],pred_labels[i][j]]
            writeline = " ".join(newarray)
            #print(writeline)
            output.write(writeline+"\n")
            counter+=1
     